# AutoEncoders

## Importing the libraries

In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset

In [ ]:
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

## Preparing the training set and the test set

In [6]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

## Getting the number of users and movies

In [7]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

## Converting the data into an array with users in lines and movies in columns

In [8]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors

In [9]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Creating the architecture of the Neural Network

In [10]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

## Training the SAE

In [15]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.item()*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: 0.9122962864609544
epoch: 2 loss: 0.9124135945514019
epoch: 3 loss: 0.9124813657254329
epoch: 4 loss: 0.9120147371911713
epoch: 5 loss: 0.9119967411464017
epoch: 6 loss: 0.9116074472495177
epoch: 7 loss: 0.9117959214113336
epoch: 8 loss: 0.911300088247584
epoch: 9 loss: 0.9110865742452618
epoch: 10 loss: 0.9107010225890773
epoch: 11 loss: 0.9110042379495668
epoch: 12 loss: 0.9103630650769328
epoch: 13 loss: 0.9105526727581045
epoch: 14 loss: 0.9096652587664943
epoch: 15 loss: 0.9100089349689517
epoch: 16 loss: 0.9093712295506277
epoch: 17 loss: 0.9092595420089462
epoch: 18 loss: 0.9089413490727202
epoch: 19 loss: 0.908944464885167
epoch: 20 loss: 0.90857142125581
epoch: 21 loss: 0.9083663593766105
epoch: 22 loss: 0.9080955844324617
epoch: 23 loss: 0.9077873952699266
epoch: 24 loss: 0.9071130110661838
epoch: 25 loss: 0.9073809633910013
epoch: 26 loss: 0.9068681352492772
epoch: 27 loss: 0.907206611227186
epoch: 28 loss: 0.9065351236010926
epoch: 29 loss: 0.9071377399104554

## Testing the SAE

In [16]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9588)


In [14]:
id_user_recommendation = 0

# Prepare the input for the selected user
input_recommendation = Variable(training_set[id_user_recommendation]).unsqueeze(0) # or torch.Tensor if not using Variable
# or if not using Variable: input_recommendation = training_set[id_user_recommendation].unsqueeze(0)


# Get the predicted ratings from the trained SAE
output_recommendation = sae(input_recommendation) # or sae(input_recommendation) if not using Variable
predicted_ratings = output_recommendation.data.numpy()[0] # or output_recommendation.detach().numpy()[0] if not using Variable


# Mask movies already rated by the user (set predicted rating to a very small value)
rated_movies_mask = training_set[id_user_recommendation].numpy() > 0 # or training_set[id_user_recommendation].numpy() > 0 if not using Variable
predicted_ratings[rated_movies_mask] = -np.inf  # Mask already rated movies


# Get the indices of movies with the highest predicted ratings (recommend top 10 movies)
top_n = 10
recommended_movie_indices = np.argsort(predicted_ratings)[::-1][0:top_n]

print("Recommendations for user", id_user_recommendation)
for movie_index in recommended_movie_indices:
    print(f"Movie Index: {movie_index}")

Recommendations for user 0
Movie Index: 1499
Movie Index: 1598
Movie Index: 1641
Movie Index: 1448
Movie Index: 407
Movie Index: 1466
Movie Index: 1652
Movie Index: 850
Movie Index: 1638
Movie Index: 1366
